In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')

In [2]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

hf_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

/Users/amarmandal/Documents/coding/GenAI_Course/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
len(hf_embeddings.embed_query("This is a test query"))

384

In [6]:
# openai embeddings
from langchain_openai import OpenAIEmbeddings

openai_emb_obj = OpenAIEmbeddings()

openai_embedding = openai_emb_obj.embed_query("This is a test query")
len(openai_embedding)

1536

#### Pinecone

In [9]:
from pinecone import Pinecone
import os

os.environ['PINECONE_API_KEY'] = os.getenv('PINECONE_API_KEY')


In [10]:
pc = Pinecone(
    api_key=os.getenv('PINECONE_API_KEY'),
)

### Creating a Pinecone index

In [14]:
from pinecone import ServerlessSpec

index_name = "genaicourse"

if not pc.has_index(index_name): 
    pc.create_index(
        name=index_name,
        dimension=1536,  # Dimension of the embeddings
        metric="cosine",  # Similarity metric
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1",
        )
    )

### LOading data into Pinecone

In [15]:
from langchain_pinecone import PineconeVectorStore

vs = PineconeVectorStore(
    index_name=index_name,
    embedding=openai_emb_obj
)

In [16]:
from uuid import uuid4
from langchain_core.documents import Document

# 10 documents to add
docs = [
    Document(
        page_content=f"This is a test document {i}",
        metadata={
            "id": str(uuid4()),
            "source": f"source_{i}",
            "title": f"Title {i}",
            "author": f"Author {i}"
        }
    )
    for i in range(10)
]

In [17]:
# adding documents to the vector store
vs.add_documents(docs)

['daf22ea2-3aca-43e5-874a-8a94adf3feee',
 '92ad00a4-ef5a-436e-80f5-3d9979625044',
 '213f3470-4031-477b-ae04-b9d26bb85228',
 'f5978339-7ad8-43a6-bf33-33af70b6fff7',
 '4ec7c629-d3e1-4f8c-8b38-135963d8359a',
 'cb72af4c-5131-4951-bc48-907ab84ee8b9',
 '1b102b6c-5f93-4869-846f-c5bd8d357360',
 '1481a4c9-33b3-40ae-b106-8b1c5f15a764',
 '7358ef83-b010-42bb-aa41-006041e137b9',
 '0f7d3fc3-3f2c-462d-88dc-38238726530f']

In [19]:
results = vs.similarity_search("This is a test query", k=3, filter={"author": "Author 1"})
for result in results:
    print(f"Document: {result.page_content}, Metadata: {result.metadata}")

Document: This is a test document 1, Metadata: {'author': 'Author 1', 'id': 'c1b8b8f5-cc00-45a8-b038-68b213912194', 'source': 'source_1', 'title': 'Title 1'}


In [26]:
retriever = vs.as_retriever(
    search_type="similarity_score_threshold",  # Using similarity search
    search_kwargs={
        "k": 3,
        "score_threshold": 0.5,  # Optional threshold for filtering results
    }
)

# Using the retriever to get documents
retrieved_docs = retriever.invoke("This is a test query")

In [27]:
retrieved_docs

[Document(id='92ad00a4-ef5a-436e-80f5-3d9979625044', metadata={'author': 'Author 1', 'id': 'c1b8b8f5-cc00-45a8-b038-68b213912194', 'source': 'source_1', 'title': 'Title 1'}, page_content='This is a test document 1'),
 Document(id='f5978339-7ad8-43a6-bf33-33af70b6fff7', metadata={'author': 'Author 3', 'id': '201b0c8a-5737-4ac5-8aba-b3016dd239d7', 'source': 'source_3', 'title': 'Title 3'}, page_content='This is a test document 3'),
 Document(id='213f3470-4031-477b-ae04-b9d26bb85228', metadata={'author': 'Author 2', 'id': 'f434d7ec-be8f-4da8-8dab-03dc5bdfc161', 'source': 'source_2', 'title': 'Title 2'}, page_content='This is a test document 2')]

#### RAG

In [28]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini"
)

In [31]:
from langchain import hub

os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')

prompt  = hub.pull("rlm/rag-prompt")

In [40]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [41]:
from langchain_core.prompts import PromptTemplate

rag_prompt = PromptTemplate(
    template = "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:",
    input_variables=["context", "question"]
)

In [42]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [43]:
rag_chain.invoke("Total records")

'The total number of records is 3.'